<a href="https://colab.research.google.com/github/jlc2295/DSIRiverProject/blob/master/1_6_Use_Of_Zero_Crossing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=JNEvYtiw-DbwWch7OVO9h7BY33S3dYyCCVnxOIDApZE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3gEurnmkR8KE2Phf7jh7pieQRKt9wVl9hC7uprBK9qNJ384oSAVNSXs

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
def get_images(path_list , row_list , satelite , start_date , end_date , max_cloud_percentage , months):
    
    # get image collection object
    coll = ee.ImageCollection(satelite)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.inList('WRS_PATH', path_list))\
        .filter(ee.Filter.inList('WRS_ROW', row_list))\
        .filter(ee.Filter.lt('CLOUD_COVER' , max_cloud_percentage))\
        .filter(ee.Filter.calendarRange(months[0],months[1],'month'))    # just may data
    # get image_id's
    image_ids = list( map( lambda x : x['id'] , coll.getInfo()['features'] ) ) 
    
    # get image objects
    images = list( map( lambda x: ee.Image(x) , image_ids ) )
    
    return images

In [5]:
#Lake Chapala Region
p = [29]
r = [46]
sat = 'LANDSAT/LC08/C01/T1_TOA'
sd = '2013-05-01'
ed = '2020-05-01'
cc= 10
months = [5 , 5] 
image_list = get_images(p, r, sat, sd, ed, cc, months)

In [6]:
'''
Functions needed to display images and features/featurecollections are given here. 
These functions are all from other sources. Mostly from google tutorials.
'''
!pip install geojson
!pip install pygeoj
import geojson
import json
import pygeoj
import numpy as np

import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

  Created wheel for pygeoj: filename=PyGeoj-1.0.0-cp36-none-any.whl size=10349 sha256=97b990233a1dd7bad540b333d37d8141f796e515c4b8ea69678b9fb5bb185bb5
  Stored in directory: /root/.cache/pip/wheels/7d/7c/8e/3c82fc15ee9f04d8df3e59817d2ee5499a8413341d6476c7d8
Successfully built pygeoj


In [7]:
image = image_list[0]
image = image.select('B7')
canny = ee.Algorithms.CannyEdgeDetector(image, 10, 1)

# Create a folium map object.
my_map = folium.Map(location=[20.08, -103.16], zoom_start=10, height=500)
my_map.add_ee_layer(image, {max: 12000},'image')

display(my_map)

**Applying zeroCrossing**

Google API Guide: https://developers.google.com/earth-engine/guides/image_edges

A zero-crossing is defined as any pixel where the right, bottom, or diagonal bottom-right pixel has the opposite sign. If any of these pixels is of opposite sign, the current pixel is set to 1 (zero-crossing); otherwise it's set to zero. To detect edges, the zero-crossings algorithm can be applied to an estimate of the image second derivative. 


In [8]:
# Define a "fat" Gaussian kernel.
fat = ee.Kernel.gaussian(3, 3,'pixels',bool('true'),-1)

# Define a "skinny" Gaussian kernel.
skinny = ee.Kernel.gaussian(3,1,'pixels',bool('true'))

#Compute a difference-of-Gaussians (DOG) kernel.
dog = fat.add(skinny)

#Compute the zero crossings of the second derivative, display.
zeroXings = image.convolve(dog).zeroCrossing()

my_map.add_ee_layer(zeroXings.updateMask(zeroXings), {'palette': 'FF0000'}, 'zero crossings')

display(my_map)

With zeroCrossing, you can see that the method is very non-discriminatory: you can faintly see the outline of the lake amongst all the red lines.

Let's try to use Canny Edge Detection next.